# Imports

In [1]:
import os
import csv
import numpy as np
import pandas as pd
from modules.misc.glicko2 import calculate_updated_glicko2
from modules.misc.model_path_handling import create_model_dictionary_from_path

# Constants

In [2]:
GAME_RESULTS_FILE_NAME = 'game_results.csv'
RATING_HISTORY_FILE_NAME = 'rating_history.csv'
GAME_RESULTS_PATH = f'c:\\repos\\Airhockey_Agent_Weights\\tournaments\\fromScratch\\{GAME_RESULTS_FILE_NAME}'
RATING_HISTORY_PATH = f'c:\\repos\\Airhockey_Agent_Weights\\tournaments\\fromScratch\\{RATING_HISTORY_FILE_NAME}'
# init glicko2
RATING = 1500
RD = 350
VOLATILITY = 0.06
TAU = 0.5

# Calculation

In [ ]:
# Read the CSV data from the game results file
game_results = pd.read_csv(GAME_RESULTS_PATH)
# create empty dataframe for current rating period
rating_period = pd.DataFrame(columns=["game_id","player_key_a", "player_key_b", "score_a", "score_b","rating_period"])
rating_histories = pd.DataFrame(columns=["player_key","rating","rating_deviation","volatility"])

# add initial ratings to rating history
rating_histories = rating_histories.append({'player_key': '230426_092040_SAC_Airhockey_Selfplay_FromScratch_24h_059093', 'rating': RATING, 'rating_deviation': RD, 'volatility': VOLATILITY}, ignore_index=True)
# rating_histories = rating_histories.append({'player_key': '230406_154802_SAC_Airhockey_Selfplay_DomainRandomizedContinued_054731', 'rating': rating, 'rating_deviation': rating_deviation, 'volatility': volatility}, ignore_index=True)
rating_histories = rating_histories.append({'player_key': '230426_092040_SAC_Airhockey_Selfplay_FromScratch_48h_113147', 'rating': RATING, 'rating_deviation': RD, 'volatility': VOLATILITY}, ignore_index=True)
rating_histories = rating_histories.append({'player_key': '230426_092040_SAC_Airhockey_Selfplay_FromScratch_72h_174208', 'rating': RATING, 'rating_deviation': RD, 'volatility': VOLATILITY}, ignore_index=True)
rating_histories = rating_histories.append({'player_key': '230429_124236_SAC_Airhockey_Selfplay_FromScratch_Continued_24h_058521', 'rating': RATING, 'rating_deviation': RD, 'volatility': VOLATILITY}, ignore_index=True)
rating_histories = rating_histories.append({'player_key': '230429_124236_SAC_Airhockey_Selfplay_FromScratch_Continued_48h_058521', 'rating': RATING, 'rating_deviation': RD, 'volatility': VOLATILITY}, ignore_index=True)

# get how many rating periods there are in the game results file
amount_rating_periods = game_results['rating_period'].unique().size
# loop through each rating period
for i in range(amount_rating_periods):
    # append all games of the current rating period to the rating period dataframe
    rating_period = rating_period.append(game_results.loc[game_results['rating_period'] == i])
    # get unique agent names of the current rating period for player key a and b to list
    unique_agents = np.unique(rating_period[['player_key_a', 'player_key_b']].values)
    # create for each agent a game history dataframe for the current rating period
    agent_game_histories = {}
    agent_game_history_df = pd.DataFrame(columns=['game_id', 'opponent','score', 'opponent_score', 'rating_self', 'rating_deviation_self','volatility_self', 'rating_opponent', 'rating_deviation_opponent','volatility_opponent'])
    
    # calculate new ratings based on tourney results
    for agent in unique_agents:        
        for _, game in rating_period.iterrows():            
            # get agent names
            agent_a = game['player_key_a']
            agent_b = game['player_key_b']
            
            # get agent ratings
            # make sure rating histories is dataframe            
            rating_a = rating_histories.loc[rating_histories['player_key'] == agent_a].iloc[-1]['rating']
            rating_b = rating_histories.loc[rating_histories['player_key'] == agent_b].iloc[-1]['rating']
            # get agent rating deviations
            rating_deviation_a = rating_histories.loc[rating_histories['player_key'] == agent_a].iloc[-1]['rating_deviation']
            rating_deviation_b = rating_histories.loc[rating_histories['player_key'] == agent_b].iloc[-1]['rating_deviation']
            # get agent volatility
            volatility_a = rating_histories.loc[rating_histories['player_key'] == agent_a].iloc[-1]['volatility']
            volatility_b = rating_histories.loc[rating_histories['player_key'] == agent_b].iloc[-1]['volatility']
            # get agent scores
            score_a = game['score_a']
            score_b = game['score_b']
            # determine game id for agent - if agent has no game history, game id is 0, otherwise it is the last game id + 1
            if agent_a in agent_game_histories:
                game_id_a = agent_game_histories[agent_a].iloc[-1]['game_id'] + 1
            else:
                game_id_a = 0
            if agent_b in agent_game_histories:
                game_id_b = agent_game_histories[agent_b].iloc[-1]['game_id'] + 1
            else:
                game_id_b = 0
                
            if agent == agent_a:    
                new_row = {
                    'game_id': game_id_a,
                    'opponent': agent_b,
                    'score': score_a,
                    'opponent_score': score_b,
                    'rating_self': rating_a,
                    'rating_deviation_self': rating_deviation_a,
                    'volatility_self': volatility_a,
                    'rating_opponent': rating_b,
                    'rating_deviation_opponent': rating_deviation_b,
                    'volatility_opponent': volatility_b
                }
                if not agent_a in agent_game_histories:
                    agent_game_histories[agent_a] = agent_game_history_df
                agent_game_histories[agent_a] = agent_game_histories[agent_a].append(new_row, ignore_index=True)

            if agent == agent_b:
                new_row = {
                    'game_id': game_id_b,
                    'opponent': agent_a,
                    'score': score_b,
                    'opponent_score': score_a,
                    'rating_self': rating_b,
                    'rating_deviation_self': rating_deviation_b,
                    'volatility_self': volatility_b,
                    'rating_opponent': rating_a,
                    'rating_deviation_opponent': rating_deviation_a,
                    'volatility_opponent': volatility_a
                }
                if not agent_b in agent_game_histories:
                    agent_game_histories[agent_b] = agent_game_history_df
                agent_game_histories[agent_b] = agent_game_histories[agent_b].append(new_row, ignore_index=True)
    # calculate new ratings based on game results for each agent
    for agent_name, game_history_df in agent_game_histories.items():
        # get current agent rating
        rating = rating_histories.loc[rating_histories['player_key'] == agent_name].iloc[-1]['rating']
        # get current agent rating deviation
        rating_deviation = rating_histories.loc[rating_histories['player_key'] == agent_name].iloc[-1]['rating_deviation']
        # get current agent volatility
        volatility = rating_histories.loc[rating_histories['player_key'] == agent_name].iloc[-1]['volatility']

        # calculate new rating
        rating_updated, rating_deviation_updated, volatility_updated = calculate_updated_glicko2(rating=rating, rating_deviation=rating_deviation, volatility=volatility, opponents_in_period=game_history_df, tau=TAU, rd_threshold=30)
        # add new rating to rating history
        rating_histories = rating_histories.append({'player_key': agent_name, 'rating': rating_updated, 'rating_deviation': rating_deviation_updated, 'volatility': volatility_updated}, ignore_index=True)
        # if csv file does not exist, create it and add header
        if not os.path.isfile(RATING_HISTORY_PATH):
            with open(RATING_HISTORY_PATH, 'w', newline='') as file:
                # csv writer
                writer = csv.writer(file)
                # write header
                writer.writerow(['player_key', 'rating', 'rating_deviation', 'volatility'])
        # open csv file to append new ratings for current agent
        with open(RATING_HISTORY_PATH, 'a', newline='') as file:
            # csv writer
            writer = csv.writer(file)
            # write new ratings at the end of the csv file without overwriting existing ratings
            writer.writerow([agent_name, rating_updated, rating_deviation_updated, volatility_updated])
    
    # reset rating period dataframe
    rating_period = pd.DataFrame(columns=['rating_period', 'player_key_a', 'player_key_b', 'score_a', 'score_b'])